Input data represents 1 ping to 8.8.8.8 performed every minute, 24/7

This notebook collects and transforms data to present periods of failing pings, which represent downtime, or at least degraded performance.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dataset
import datetime
from collections import Counter

In [2]:
ds = dataset.load_dataset('googledns.log')
len(ds)

591341

In [3]:
success_count = sum(map(lambda datapoint: datapoint[1], ds))
fail_count = len(list(filter(lambda datapoint: datapoint[1] == 0, ds)))
print("Successful:", success_count)
print("Failures  :", fail_count)
print("Total     :", (success_count + fail_count))

Successful: 587217
Failures  : 4124
Total     : 591341


In [4]:
# rough estimate of the number of hours of not being able to ping google
print("Estimated hour of failure:", (fail_count / 60))

Estimated hour of failure: 68.73333333333333


In [5]:
# Let's find when things went wrong

# find points at which is_good(previous) XOR is_good(current)
changepoints = [(0, ds[0][0], 0)]
for i in range(len(ds) - 1):
    data, received = ds[i]
    if received ^ ds[i+1][1]:
        changepoints.append((i+1, ds[i+1][0], ds[i+1][1])) #idx, timestamp, status

In [6]:
outages = []
for i in range(len(changepoints)):
    idx, start, status = changepoints[i]
    if status == 0:
        if i+1 < len(changepoints):
            end = changepoints[i+1][1]
            next_idx = changepoints[i+1][0]
        else:
            end = datetime.datetime()
            next_idx = -1
        duration = (end - start)
        outages.append((start, duration, next_idx - idx))

# discard single ping failures, only use consecutive ping failures as indicator of reduced connectivity
outages = list(filter(lambda o: o[2] > 1, outages))

In [7]:
print("Last 20 consecutive ping fails:")

for outage in outages[-20:]:
    start, duration, samples = outage

    print("No successful pings to 8.8.8.8 from %s: %s (# failed: %s)"%(start, duration, samples))

Last 20 consecutive ping fails:
No successful pings to 8.8.8.8 from 2021-08-06 16:22:01: 0:20:59 (# failed: 21)
No successful pings to 8.8.8.8 from 2021-08-06 16:49:01: 0:13:00 (# failed: 13)
No successful pings to 8.8.8.8 from 2021-08-06 17:07:01: 0:13:00 (# failed: 13)
No successful pings to 8.8.8.8 from 2021-08-06 17:26:01: 0:33:01 (# failed: 33)
No successful pings to 8.8.8.8 from 2021-08-09 11:46:01: 0:09:00 (# failed: 9)
No successful pings to 8.8.8.8 from 2021-08-09 12:26:01: 0:03:00 (# failed: 3)
No successful pings to 8.8.8.8 from 2021-08-09 22:40:01: 0:43:59 (# failed: 44)
No successful pings to 8.8.8.8 from 2021-08-09 23:30:01: 0:48:00 (# failed: 48)
No successful pings to 8.8.8.8 from 2021-08-10 00:26:01: 0:47:00 (# failed: 47)
No successful pings to 8.8.8.8 from 2021-08-10 01:24:01: 0:43:00 (# failed: 43)
No successful pings to 8.8.8.8 from 2021-08-10 02:13:01: 0:48:00 (# failed: 48)
No successful pings to 8.8.8.8 from 2021-08-10 03:10:02: 0:44:59 (# failed: 45)
No success

In [8]:
outage_counter = {}

for outage in outages:
    start, duration, samples = outage
    
    if samples < 2: continue

    year_month = "%d-%02d"%(start.year, start.month)
    updated_total = outage_counter.setdefault(year_month, 0) + samples
    outage_counter[year_month]=updated_total
    
for ym in sorted(outage_counter.keys()):
    pingfails = outage_counter[ym]
    print("Total missed pings to 8.8.8.8 on %s: %5d (%.2f hours of degraded connectivity)"%(ym, pingfails, pingfails / 60))

Total missed pings to 8.8.8.8 on 2020-06:   651 (10.85 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2020-07:    49 (0.82 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2020-08:  2199 (36.65 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2020-09:     8 (0.13 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2020-10:     2 (0.03 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2020-11:     9 (0.15 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2020-12:     7 (0.12 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2021-01:    98 (1.63 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2021-02:    18 (0.30 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2021-03:     9 (0.15 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2021-04:     8 (0.13 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2021-05:    15 (0.2